In [ ]:
import pickle
import seaborn as sns
import imageio as io
import pandas as pd
import os
import matplotlib.pyplot as plt
import numpy as np
import tifffile
from tqdm.notebook import tqdm
import pathlib
import json
import glob
import PIL
import scanpy as sc

In [ ]:
adata_whole= sc.read(r'data\240719AnalysisDAM_TERM\adata_del2.h5ad')

In [ ]:
sc.pl.umap(adata_whole, color=['res3','predicted_label'],legend_loc='on data')

In [ ]:
adata_mg=adata_whole[adata_whole.obs['res3']==('3')]

In [ ]:
adata=adata_mg.copy()

In [ ]:
sc.pp.pca(adata)
sc.pp.neighbors(adata)
sc.tl.umap(adata,random_state=9,min_dist=0.001)
sc.pl.umap(adata, color='Tmem119')

In [ ]:
sc.tl.leiden(adata,resolution=2, key_added='mg_res2')
sc.pl.umap(adata, color=['mg_res2'], legend_loc='on data')

In [ ]:
sc.set_figure_params(figsize=(5,5))
sc.pl.umap(adata, color=['mg_res2','Flt1','Aqp4','Olig1'], legend_loc='on data')

In [ ]:
cluster_mapping = adata.obs['mg_res2'].to_dict()

adata_whole.obs['mg_res2'] = adata_whole.obs_names.map(cluster_mapping)
adata_whole.obs['mg_res2'].fillna('NotClustered', inplace=True)

In [ ]:
clusters=['9','3','21','6']
adata=adata[~adata.obs['mg_res2'].isin(clusters)]
adata

In [ ]:
sc.pp.pca(adata)
sc.pp.neighbors(adata)
sc.tl.umap(adata,random_state=9,min_dist=0.001)
sc.pl.umap(adata, color='Tmem119')

In [ ]:
sc.tl.leiden(adata,resolution=1, key_added='mg2_res1')
sc.tl.leiden(adata,resolution=0.5, key_added='mg2_res0p5')
sc.tl.leiden(adata,resolution=2, key_added='mg2_res2')
sc.tl.leiden(adata,resolution=1.5, key_added='mg2_res1p5')

In [ ]:
sc.pl.umap(adata, color=['mg2_res1','mg2_res0p5','mg2_res2','mg2_res1p5'], legend_loc='on data')

In [ ]:
clusters=['3']
adata_tr=adata[~adata.obs['mg2_res1p5'].isin(clusters)]
adata_tr

In [ ]:
sc.pp.pca(adata_tr)
sc.pp.neighbors(adata_tr)
sc.tl.umap(adata_tr,random_state=9,min_dist=0.001)
sc.pl.umap(adata_tr, color='Tmem119')

In [ ]:
sc.tl.leiden(adata_tr,resolution=1, key_added='mgtr_res1')
sc.tl.leiden(adata_tr,resolution=0.5, key_added='mgtr_res0p5')
sc.tl.leiden(adata_tr,resolution=2, key_added='mgtr_res2')
sc.tl.leiden(adata_tr,resolution=1.5, key_added='mgtr_res1p5')

In [ ]:
sc.pl.umap(adata_tr, color=['mgtr_res1','mgtr_res0p5','mgtr_res2','mgtr_res1p5'], legend_loc='on data')

In [ ]:
adata_tr.obs['mgtr_res1p5'].value_counts()

In [ ]:
sc.set_figure_params(figsize=(5,5))
sc.pl.umap(
    adata_tr, 
    color=['mgtr_res1'],
    add_outline=True,
    legend_loc='on data', #right margin
    legend_fontsize=15,
    legend_fontoutline=4,
    frameon=False,
    size=20,
    show=False,
    use_raw=False,
    cmap='coolwarm'
)

In [ ]:
cluster_mapping = adata_tr.obs['mgtr_res1p5'].to_dict()
adata_whole.obs['mgtr_res1p5'] = adata_whole.obs_names.map(cluster_mapping)
adata_whole.obs['mgtr_res1p5'].fillna('NotClustered', inplace=True)

In [ ]:
def plot_cluster_scdata_str(scdata, cmap, clusters=[1, 2], transpose=1, flipx=1, flipy=1, tag='cluster', key='X_spatial'):
    import matplotlib.pyplot as plt
    import numpy as np
    
    # Generate unique color mappings for clusters
    unique_clusters = np.unique(scdata.obs[tag])
    cluster_to_color = {str(cluster): cmap[i % len(cmap)] for i, cluster in enumerate(unique_clusters)}
    
    x, y = (np.array(scdata.obsm[key]) * [flipx, flipy])[:, ::transpose].T
    plt.scatter(x, y, c='grey', s=4, marker='.')
    
    for cluster in clusters:
        cluster_ = str(cluster)
        inds = scdata.obs[tag] == cluster_
        x_ = x[inds]
        y_ = y[inds]
        col = cluster_to_color.get(cluster_, 'grey')  # Get the color from the mapping
        plt.scatter(x_, y_, c=col, s=15, marker='.', label=cluster_)
    
    plt.grid(False)
    plt.axis("off")
    plt.axis("equal")
    plt.legend()
    plt.tight_layout()
    return plt.gcf()

In [ ]:
clusters = adata_whole.obs['mgtr_res1p5'].unique()
output_folder = r'data\240719AnalysisDAM_TERM\f4_adata_whole_mgtr_res1p5_single_cluster_spxMap'
for cluster in clusters:
    fig = plt.figure(figsize=(12, 8), facecolor="black")
    plot_cluster_scdata_str(adata_whole, cmap, clusters=[cluster], transpose=1, flipx=1, flipy=-1, tag='mgtr_res1p5', key='X_multi_spatial')
    
    plt.savefig(f'{output_folder}\\f4_cluster_{cluster}_whole_mgtr_res1.jpg', format='jpg', dpi=300, bbox_inches='tight')
    
    plt.close(fig)

In [ ]:
adata_tr.uns['log1p']["base"] = None
sc.tl.rank_genes_groups(adata_tr, 'mgtr_res1p5', method='t-test')
result = adata_tr.uns['rank_genes_groups']
groups = result['names'].dtype.names
df_scdata = pd.DataFrame({group + '_' + key[:1]: result[key][group] for group in groups for key in ['names','logfoldchanges','pvals','pvals_adj']})
df_scdata.to_csv('f4_adata_tr_mgres1p5_rgg.csv')

In [ ]:
plt.style.use('default')
sc.tl.rank_genes_groups(adata_tr, groupby='mgtr_res1p5', groups=['1','6','4','13'], reference='0', method='wilcoxon')
sc.pl.rank_genes_groups_violin(adata_tr, n_genes=20)
#sc.pl.rank_genes_groups_stacked_violin(adata_tr, n_genes=20)

In [ ]:
sc.tl.rank_genes_groups(adata_tr, groupby='mgtr_res1p5', groups=['7','8','14','11','15','9'], reference='0', method='wilcoxon')
sc.pl.rank_genes_groups_violin(adata_tr, n_genes=20)

In [ ]:
plt.style.use('default')
sc.tl.rank_genes_groups(adata_tr, groupby='mgtr_res1p5', groups=['5','12','2','10'], reference='0', method='wilcoxon')
sc.pl.rank_genes_groups_violin(adata_tr, n_genes=20)

In [ ]:
fig=plt.figure(figsize=(12, 8), facecolor="black")
fig = plot_cluster_scdata(adata_whole,cmap,clusters=[2,5,12],transpose=1,flipx=1,flipy=-1,tag='mgtr_res1p5', key = 'X_multi_spatial')

In [ ]:
adata_tr.uns['log1p']["base"] = None
sc.tl.rank_genes_groups(adata_tr, 'mgtr_res1p5', method='t-test')
# result = adata_tr.uns['rank_genes_groups']
# groups = result['names'].dtype.names

In [ ]:
markers=['P2ry12','Trem2','Cd9','Clec7a','Axl','Lpl','Spp1','C3','Gpnmb','H2-Aa']
sc.pl.dotplot(adata_tr, markers, groupby='mgtr_res1p5',dendrogram=True)

In [ ]:
cell_dict2={
'TERM': ['8','3','6','1'],
'DAM': ['7','11','15','9','4','14'],
'Homeo_1': ['0','10'],
'Homeo_2':['2', '12','5'],
'MHCII':['13']
}

adata_tr.obs['main_type'] = np.nan

for i in cell_dict2.keys():
    ind = pd.Series(adata_tr.obs['mgtr_res1p5']).isin(cell_dict2[i])
    adata_tr.obs.loc[ind,'main_type'] = i

In [ ]:
adata=adata_tr[adata_tr.obs['celltype_3']!='Homeo_2']

In [ ]:
sc.pp.pca(adata)
sc.pp.neighbors(adata)
sc.tl.umap(adata,random_state=9,min_dist=0.001)

In [ ]:
sc.pl.umap(
    adata, 
    color=['mgtr_res1p5','geno_type','App','Trem2','celltype_3'],
    add_outline=True,
    legend_loc='on data', #right margin
    legend_fontsize=10,
    legend_fontoutline=4,
    frameon=False,
    size=20,
    show=False,
    use_raw=False,
    cmap='coolwarm'
)

In [ ]:
cell_dict2={
'Homeo': ['Homeo_1'],
'int_DAM': ['Prime_DAM_1'],
'DAM_2':['DAM_1'],
'DAM_1': ['Prime_DAM_2'],
'int_TERM': ['Prime_TERM'],
'TERM_1':['TERM_1'],
'TERM_2':['TERM_2'],
'MHCII':['MHCII']
}

adata.obs['celltype_4'] = np.nan

for i in cell_dict2.keys():
    ind = pd.Series(adata.obs['celltype_3']).isin(cell_dict2[i])
    adata.obs.loc[ind,'celltype_4'] = i

In [ ]:
sc.pl.umap(
    adata, 
    color=['celltype_4'],
    add_outline=True,
    legend_loc='on data',
    legend_fontsize=8,
    legend_fontoutline=6,
    frameon=False,
    size=20,
    show=False,
    use_raw=False,
)

In [ ]:
cell_dict2={
'Mg_0': ['Homeo'],
'Mg_1': ['int_DAM'],
'Mg_2': ['int_TERM'],
'Mg_3': ['TERM_1'],
'Mg_4': ['DAM_2'],
'Mg_5': ['DAM_1'],
'Mg_6': ['TERM_2'],
'Mg_7': ['MHCII']
}

adata.obs['celltype_5'] = np.nan

for i in cell_dict2.keys():
    ind = pd.Series(adata.obs['celltype_4']).isin(cell_dict2[i])
    adata.obs.loc[ind,'celltype_5'] = i

In [ ]:
custom_colors = {
    'Mg_0': 'skyblue',    # Yellow#ffe119
    'Mg_2': '#f58231',    # blue4363d8
    'Mg_1': '#ffe119', #  Green#3cb44b
    'Mg_6': '#e6194B', # Lime#3cb44b
    'Mg_5': '#f032e6', # Bright Red
    'Mg_7': '#fabed4', #  Orange#f58231 Magenta#f032e6 pink#fabed4
    'Mg_3': '#3cb44b', # Olive#808000 4363d8
    'Mg_4': '#4363d8' # Magenta#f032e6 Bright Red#e6194B     
}

unique_classes = adata.obs['celltype_5'].cat.categories
adata.uns['celltype_5_colors'] = [custom_colors.get(cls, '#000000') for cls in unique_classes]  # Default black for missing classes
sc.set_figure_params(figsize=(5, 5))
sc.pl.umap(
    adata, 
    color='celltype_5',
    add_outline=True,
    legend_loc='on data',  # Legend position
    legend_fontsize=16,
    legend_fontoutline=2,
    frameon=False,
    size=20,
    show=False,
    use_raw=False
)

output_path = f'{output_folder}/MG_umap_unicolor_update2.pdf'
plt.savefig(output_path, bbox_inches='tight', dpi=300)
plt.show()